<a href="https://colab.research.google.com/github/GUPTA-NAMAN/Reinforcement-Learning/blob/A2/Q6_Policy_Iteration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [110]:
import numpy
import random
import copy

In [131]:
# Defining data structure for state-value function("value") index (i,j)["which represent state"] represent state value of BLOCK at Co-Ordinate (i,j)
# Defining data structure for state-action function("action") at index (i,j)["which represent state"] we take action action(i,j)("which is denoted by integer for more INFO Look First-most("find_next_block_CoOrdinate") function")
value = numpy.zeros( (4,4) , "float" )           # STATE-VALUE
action = numpy.zeros( (4,4) ,"int" )             # POLICY
gamma = 0.9                                      # Dis-counting factor

In [132]:
# Randomly initializing state-action
for i in range(4) :
  for j in range(4) :
    action[i][j] = random.uniform(1,4) 
action[0][0] = -1   # NO defined action because it is terminating state
action[3][3] = -1

In [113]:
# It takes current position and action as input and output co_Ordunate after taking action 
# action==1 ; UP
# action==2 ; Down
# action==3 ; Left
# action==4 ; Rigth
def find_next_block_CoOrdinate(action,i,j) :
  if action == 1 :
    return (i-1,j) 
  elif action == 2 :
    return (i+1,j) 
  elif action == 3 :
    return (i,j-1) 
  else :
    return (i,j+1) 


In [114]:
# It check if after taking ACTION "action" in BLOCK (i,j)["either state"] . wheather the new position is in Grid or NOT
def Is_IN_Grid_After_taking_action(action,i,j) :
  new_i,new_j = find_next_block_CoOrdinate(action,i,j) 
  if new_i<0 or new_i>3 or new_j<0 or new_j>3 : 
    return False 
  else :
    return True 

In [115]:
# INPUT   :  matrix("value"),ACTION("action") , Current Block (i,j)[" either state "]
# OUTPUT  :  updated state-value 
def update(value,action,i,j) :
  sum = 0
  new_i,new_j = find_next_block_CoOrdinate(action,i,j) 
  if new_i==0 and new_j==0 :           # checking if we entered top-left terminating state
    sum = 0 
  elif new_i==4 and new_j==4 :         # checking if we entered bottom-rigth terminating state
    sum = 0
  else :
    sum  = sum + (-1)
  if (j==3) :
    #print("cordinage ",i,j)
    #print("deb action ",action)
    #print("debbug sum1 ",sum)
    do = "debugg"
  Is_IN_Grid = Is_IN_Grid_After_taking_action(action,i,j) 
  if Is_IN_Grid == True :                   # checking if are in Grid
    sum =sum +  gamma*value[new_i][new_j]
    if j==3 :
      #print("db In Grid")
      do = "debug"
  else :
    sum =sum + gamma*value[i][j]
    if j==3 :
      #print("db OUT side grid")
      do = "debug"
  if j==3 :
    #print("db final sum ",sum)
    do = "debug"
  return sum 

In [116]:
def Max_Diff(prev,cur) :
  list_difference = []
  for i in range(4) :
    for j in range(4) :
      if not (  (i==0 and j==0) or (i==3 and j==3)  ) :
        list_difference.append( abs( prev[i][j] - cur[i][j] ) )
  return max(list_difference)

In [117]:
#INPUT  : STATE-VALUE("value") , PI(ACTION={1,2,3,4}|STATE={1,...14}) 
#OUTPUT : Evaluated "STATE-VALUE" ; Terminated when ration is less than 0.9 , means relativly Not much improvement ; formulae of ration is IN LAST IF CONDITION
def policy_evaluation(value,action) :
  print("---------Policy Evaluation")
  prev_value = copy.deepcopy(value)
  previous_Max_loss= None 
  loop = 0
  while(True) :
    #print("LOOP ",loop+1)
    loop=loop+1
    for i in range(4) :
      for j in range(4) :
        if not (  ( i==0 and j==0 ) or ( i==3 and j==3 ) )  :
          value[i][j] = update(value,action[i][j],i,j)
          if(j==3) :
            #print("debbuging\t",i,j,value[i][j])
            do = "debug"
        #print(i,j,value[i][j]) 
    current_Max_loss = Max_Diff(prev_value,value) 
    if previous_Max_loss == None :
      previous_Max_loss = current_Max_loss 
    else :
      if abs(current_Max_loss-previous_Max_loss) < 1.2 :
        break 

  

In [118]:
# INPUT : list of rewards and corresponding actions 
#OUPUT  : Maximum reward from the list , action for maximum reward
def Find_optimal_action_except_current_action(new_reward,new_action) :
  maximum_reward = max(new_reward)     # Getting maximum reward
  actions = []                         # List for storing action's taken for maximum rewards. Since, multiple action can give same reward(And that same is maximum TOO)
  for i in range( len(new_reward) ) :
    if maximum_reward == new_reward[i] :
      actions.append( new_action[i] )
  return random.choice(actions),maximum_reward

In [119]:
#INPUT  :  STATE-VALUE , ACTION 
#OUTPUT :  OLD STATE-ACTION MATRIX(save old state-action in value in new matrix , before improvement  )  , Improves old state-action matrix
def policy_improvement(value,actions) :
  print("--------------Doing Policy Improvement")
  print("----OLD state action")
  print(actions)
  old_state_action = copy.deepcopy(actions) 
  for i in range(4) :
    for j in range(4) :
      if not ( ( i==0 and j==0 ) or (i==4 or j==4) )  :               # Discarding terminating states for improvement
        current_optimal_action = actions[i][j]
        current_optimal_reward = update( value,action[i][j],i,j )
        new_reward_list = [] 
        new_action_list = []
        for action_code in range(1,5) :                                 # Looping over all action {1,2,3,4}
          if not ( action_code == current_optimal_action ) :          # Discarding current state-action and only taking other actions
            new_reward_list.append( update(value,action_code,i,j) )
            new_action_list.append( action_code ) 
        new_action,new_reward = Find_optimal_action_except_current_action( new_reward_list , new_action_list ) 
        if ( new_reward > current_optimal_reward ) :
          actions[i][j] = new_action 
        else :
            Do = "DO NOTHING : Current state-action is optimal" 
  return old_state_action 
        

In [120]:
def Is_policy_stable(old,cur) :
  answer = True 
  for i in range(4) :
    for j in range(4) :
      if not( old[i][j] == cur[i][j] ) :
        answer = False 
        break 
  return answer 

In [130]:
def policy_iteration(value,number_iter,actions) :
  for i in range(number_iter) :
    print("\n\n\nPOLICY ITERATION NUMBER\t",i+1)
    policy_evaluation(value,actions) 
    print("STATE-VALUE MATRIX")
    print(value)
    old_policy = policy_improvement(value,actions) 
    print("----UPDATED POLICY")
    print(action) 
    if ( Is_policy_stable(old_policy,actions) == True ) :
     print("\npolicy stabalize at Iteration ",i+1)
     break 
  

Below max_policy_iteration is hard coded can be changed.

max_policy_iteration is the number of Maximum Iterations.

```
`# This is formatted as code`
```



In [133]:
max_policy_iteration  = 10 

In [135]:
policy_iteration(value,max_policy_iteration,action)




POLICY ITERATION NUMBER	 1
---------Policy Evaluation
STATE-VALUE MATRIX
[[  0.     0.    -2.71  -2.71]
 [-10.    -1.    -1.9   -1.9 ]
 [-10.   -10.   -10.    -1.  ]
 [-10.   -10.   -10.     0.  ]]
--------------Doing Policy Improvement
----OLD state action
[[-1  3  2  2]
 [ 2  1  3  2]
 [ 1  3  3  2]
 [ 3  2  3 -1]]
----UPDATED POLICY
[[-1  3  3  2]
 [ 1  1  3  2]
 [ 1  1  4  2]
 [ 3  2  4 -1]]



POLICY ITERATION NUMBER	 2
---------Policy Evaluation
STATE-VALUE MATRIX
[[  0.     0.    -1.    -2.71]
 [  0.    -1.    -1.9   -1.9 ]
 [ -1.    -1.9   -1.9   -1.  ]
 [-10.   -10.    -1.     0.  ]]
--------------Doing Policy Improvement
----OLD state action
[[-1  3  3  2]
 [ 1  1  3  2]
 [ 1  1  4  2]
 [ 3  2  4 -1]]
----UPDATED POLICY
[[-1  3  3  3]
 [ 1  1  3  2]
 [ 1  1  4  2]
 [ 1  4  4 -1]]



POLICY ITERATION NUMBER	 3
---------Policy Evaluation
STATE-VALUE MATRIX
[[ 0.   0.  -1.  -1.9]
 [ 0.  -1.  -1.9 -1.9]
 [-1.  -1.9 -1.9 -1. ]
 [-1.9 -1.9 -1.   0. ]]
--------------Doing Policy 

In [136]:
print("Final STATE VALUE")
print(value)

Final STATE VALUE
[[ 0.   0.  -1.  -1.9]
 [ 0.  -1.  -1.9 -1.9]
 [-1.  -1.9 -1.9 -1. ]
 [-1.9 -1.9 -1.   0. ]]


In [138]:
# 1== UP , 2==DOWN , 3==LEFT , 4 ==RIGHT
print("FINAL STATE ACTION")
print(action)

FINAL STATE ACTION
[[-1  3  3  3]
 [ 1  1  3  2]
 [ 1  1  4  2]
 [ 1  4  4 -1]]
